In [ ]:
# Loading the data 
import pandas as pd
import urllib

es_url = 'http://www.stoxx.com/download/historical_values/hbrbcpe.txt'
vs_url = 'http://www.stoxx.com/download/historical_values/h_vstoxx.txt'

urllib.urlretrieve(es_url, './data/es.txt')
urllib.urlretrieve(vs_url, './data/vs.txt')


In [24]:
lines = open('./data/es.txt', 'r').readlines()
lines = [line.replace(' ', '') for line in lines]

new_file = open('./data/es50.csv', 'w')
new_file.writelines('date' + lines[3][:-1] + ';DEL' + lines[3][-1])
new_file.writelines(lines[4:])
new_file.close()

es = pd.read_csv('./data/es50.csv', index_col=0, 
                 parse_dates=True, sep=';', dayfirst=True)
del es['DEL']

vs = pd.read_csv('./data/vs.txt', index_col=0, header=2,
                 parse_dates=True, sep=',', dayfirst=True)
vs.tail()

,V2TX,V6I1,V6I2,V6I3,V6I4,V6I5,V6I6,V6I7,V6I8
Date,,,,,,,,,
2016-02-08,33.3917,34.7251,33.2189,31.8308,31.4783,30.6941,30.3705,29.8478,29.2559
2016-02-09,33.9664,35.7884,33.7800,32.6220,32.2459,31.5572,31.1659,30.5339,29.9000
2016-02-10,33.4528,34.6020,33.3637,32.1438,32.0018,31.2099,30.9937,30.8174,30.2533
2016-02-11,38.3051,41.2095,38.1347,35.8676,35.1073,33.7847,33.4126,32.2234,31.2124
2016-02-12,35.6846,35.3849,35.6965,34.1404,33.8645,32.9590,32.8208,32.3189,31.8194


In [75]:
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from sklearn import linear_model

data = pd.DataFrame({'EUROSTOXX':
                     es['SX5E'][es.index > dt.datetime(1999, 1, 1)]})
data = data.join(pd.DataFrame({'VSTOXX':
                               vs['V2TX'][vs.index > dt.datetime(1999, 1, 1)]}))

new_data = data[:-40].fillna(method='ffill')
# data.plot(subplots=True, grid=True, style='b')

rets = np.log(new_data / new_data.shift(1))
# rets.plot(subplots=True, grid=True, style='b')

xdat = rets['EUROSTOXX']
ydat = rets['VSTOXX']


model = pd.ols(y=ydat, x=xdat)

model.beta

fig1 = plt.figure()
ax1 = fig1.add_subplot(111)
ax1.plot(xdat, ydat, 'r.')

x = np.linspace(ax1.axis()[0], ax1.axis()[1] + 0.01)

ax1.plot(x, model.beta[1] + model.beta[0] * x, 'b', lw=2)
ax1.grid(True)
ax1.axis('tight')
ax1.set_xlabel('EURO STOXX 50 returns')
ax1.set_ylabel('VSTOXX returns')
plt.show()

In [57]:
?pd.ols
